# Explore numba for matrix calculations
Anne Katrine Falk, 26FEB2021

[numba](https://numba.pydata.org/) is a python package, which compiles python code into machine code.

This notebook contains exploratory test for investigating ways to make matrix calculations benefit from numba

In [ ]:
import numba
from numba import jit
import numpy as np
print(f'numba version: {numba.__version__}')
print(f'numpy version: {np.__version__}')  

In [ ]:
# Verify if CUDA toolkit is found and print the version
!nvcc -V

In [ ]:
#@jit(nopython=True)
@jit(nopython=True, parallel=True) #parallel only works on CPU, the doc says...
def numba_create_random_square_matrix(size):
    return np.random.rand(size,size)

In [ ]:
numba_create_random_square_matrix(5)

In [ ]:
size = 2000

In [ ]:
%%timeit -r 7 -n 10
x = np.random.rand(size,size)

In [ ]:
%%timeit -r 7 -n 10
x = numba_create_random_square_matrix(size)

# Several ways to calculate matrix power - with and without numba

In [ ]:
@jit(nopython=True)
def numba_matrix_power(a, p):
    """
    numba 
    Matrix power calculated using @ operator and recursion.
    """
    if p==1:
        return a
    elif p%2 == 0:
        b = numba_matrix_power(a, p//2)
        return b @ b
    else: 
        return numba_matrix_power(a, p-1) @ a

In [ ]:
def matrix_power(a, p):
    """Matrix power calculated using @ operator and recursion."""
    if p==1:
        return a
    elif p%2 == 0:
        b = matrix_power(a, p//2)
        return b @ b
    else: 
        return matrix_power(a, p-1) @ a

In [ ]:
@jit(nopython=True)
def numba_matrix_power_dot(a, p):
    """
    numba
    Matrix power calculated using np.dot
    """
    if p==1:
        return a
    elif p%2 == 0:
        b = numba_matrix_power_dot(a, p//2)
        return np.dot(b, b)
    else: 
        return np.dot(numba_matrix_power_dot(a, p-1), a)

In [ ]:
# returns an error when called, as np.matmul is not supported by numba
@jit(nopython=True)
def numba_matrix_power_matmul(a, p):
    """Matrix power calculated using np.matmul"""
    if p==1:
        return a
    else: 
        return np.matmul(numba_matrix_power_matmul(a, p-1), a)

In [ ]:
@jit(nopython=True)
def numba_linalg_matrix_power(a, p):
    """
    numba
    Matrix power calculated using np.linalg.matrix_power
    """
    return np.linalg.matrix_power(a, p)    

### check of power functions

In [ ]:
a=np.array([[2.,2.], [2.,2.]])

expect  $a^2 = \begin{bmatrix} 8 & 8 \\ 8 & 8 \end{bmatrix}$ and $a^3 = \begin{bmatrix} 32 & 32 \\ 32 & 32 \end{bmatrix}$

In [ ]:
numba_matrix_power(a, 2)

In [ ]:
numba_matrix_power_dot(a, 2)

In [ ]:
np.linalg.matrix_power(a, 3)

In [ ]:
matrix_power(a, 3)

In [ ]:
# expexted to throw an error, at numba with nopython=True does not support np.matmul
numba_matrix_power_matmul(a, 2)

# Generate a large matrix of random numbers and take a high power of it

In [ ]:
# create a square matrix of random numbers
size= 2000
x = numba_create_random_square_matrix(size)

In [ ]:
power = 50

## Time the different implementations of matrix power
Run the timing of the numba implementations twice, as a @jit marked function is compiled first time it it is called - and therefore the compilation time is included in the first timing.

### @ operator

In [ ]:
%%timeit -r 1 -n 1
numba_matrix_power(x, power)

In [ ]:
%%timeit -r 1 -n 1
matrix_power(x, power)

### np.linalg.matrix_power

In [ ]:
%%timeit -r 1 -n 1
numba_linalg_matrix_power(x, power)

In [ ]:
%%timeit -r 1 -n 1
np.linalg.matrix_power(x, power)

It is hard to see any time differences - neither between numba/non-numba implementations and np.linalg vs homebrewed recursion implementation